In [1]:
import pandas as pd
import joblib 
import numpy as np 
import datetime as datetime


In [2]:
crowdfunding = joblib.load("crowdfunding.pkl")
annual = joblib.load("annual.pkl")
managers_melt = joblib.load("managers_melt.pkl")
age = joblib.load("age.pkl")


# Managers and Age

In [3]:
ipo_stats = managers_melt.merge( age, how='left',on='ticker') 
ipo_stats

,trade_date,company_name_x,ticker,offer_price,opening_price,1st_day_close,1st_day_%_px_chng_,$_change_opening,$_change_close,star_ratings,...,cusip,offer_date,company_name_y,founding,perm,vc_dummy,rollup,dual,post-issue_shares,internet
0,1/17/20,I-Mab,IMAB,$14.00,$14.75,$12.75,-8.93%,$0.75,($1.25),2,...,44975P103,20200117.0,I-MAB,2014,19193,0,.,1,1.312394e+08,0
1,1/17/20,LIZHI,LIZI,$11.00,$11.03,$11.63,5.73%,$0.03,$0.63,1,...,53933L104,20200117.0,Lizhi Inc,2010,19195,1,.,1,9.146611e+08,0
2,1/17/20,Phoenix Tree Holdings Ltd.,DNK,$13.50,$13.50,$13.50,0.00%,$0.00,$0.00,1,...,719156101,20200117.0,Phoenix Tree Holdings Ltd,2015,19168,0,.,1,1.825797e+09,0
3,1/17/20,Velocity Financial,VEL,$13.00,$13.85,$13.51,3.92%,$0.85,$0.51,1,...,92262D101,20200117.0,Velocity Financial LLC,2004,19172,2,0,0,0.000000e+00,0
4,1/24/20,Gores Holdings IV,GHIVU,$10.00,$10.52,$10.45,4.50%,$0.52,$0.45,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9626,10/14/15,First Data,FDC,$16.00,$16.39,$15.75,-1.56%,$0.39,($0.25),1,...,319963104,19920409.0,First Data,1978,77546,1,0,0,0.000000e+00,0
9627,10/14/15,First Data,FDC,$16.00,$16.39,$15.75,-1.56%,$0.39,($0.25),1,...,32008D106,20151015.0,First Data Corp,1989,15703,0,0,1,8.790301e+08,0
9628,1/23/14,Santander Consumer USA Holdings,SC,$24.00,$25.75,$25.28,5.33%,$1.75,$1.28,2,...,80283M101,20140123.0,Santander Consumer USA Hold,1995,14414,2,0,0,0.000000e+00,0
9629,2/6/20,PPD,PPD,$27.00,$31.00,$30.00,11.11%,$4.00,$3.00,3,...,69355F102,20200206.0,PPD Inc,1985,19199,0,0,0,0.000000e+00,0


In [4]:
ipo_stats.drop(ipo_stats.columns[np.r_[6:9,10,11,13:16,17]], axis=1, inplace=True)

In [5]:
ipo_stats.dropna(inplace=True)

In [6]:
ipo_stats.rename(columns={
    'company_name_x':'company_name'},
                inplace=True)

In [7]:
ipo_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8387 entries, 0 to 9630
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   trade_date         8387 non-null   object 
 1   company_name       8387 non-null   object 
 2   ticker             8387 non-null   object 
 3   offer_price        8387 non-null   object 
 4   opening_price      8387 non-null   object 
 5   1st_day_close      8387 non-null   object 
 6   star_ratings       8387 non-null   object 
 7   managers           8387 non-null   object 
 8   founding           8387 non-null   object 
 9   vc_dummy           8387 non-null   object 
 10  rollup             8387 non-null   object 
 11  dual               8387 non-null   object 
 12  post-issue_shares  8387 non-null   float64
 13  internet           8387 non-null   object 
dtypes: float64(1), object(13)
memory usage: 982.9+ KB


In [8]:
def format_prices(df,col):
    df[col] = df[col].str.strip('$\n\t').str.replace(',','').astype(float)
    return df[col]

cols = ['offer_price', 'opening_price','1st_day_close']

for col in cols:
    format_prices(ipo_stats,col)



In [9]:
ipo_stats['trade_date'] = pd.to_datetime(ipo_stats['trade_date'], errors = 'coerce')


In [14]:
ipo_stats.dropna(inplace=True)

In [15]:
joblib.dump(ipo_stats,"ipo_stats.pkl")

['ipo_stats.pkl']

# IPO Stats and Financials

In [12]:
import re
annual['ticker'] = annual['instance'].map(lambda x: re.split('(\d+)',x))

tick = pd.DataFrame(annual['ticker'])
tick['ticker'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 4588, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[duk-, 20121231, .xml]                141
[frfd-, 20181231, .xml]               128
[frfd-, 20171231, .xml]               122
[exc-, 20161231, .xml]                120
[exc-, 20191231, x, 10, k_htm.xml]    120
                                     ... 
[bfcf-, 20151231, .xml]                 1
[zayo-, 20170630, .xml]                 1
[vktx-, 20161231, .xml]                 1
[volc-, 20121231, .xml]                 1
[netr-, 20131231, .xml]                 1
Name: ticker, Length: 8895, dtype: int64

In [13]:
new_df = annual.merge(ipo_stats, on=['company_norm'], how='left')
new_df

KeyError: 'company_norm'